In [32]:
using JuMP
using Gurobi
using LinearAlgebra

function opt_strategy_basic(tot_no_laps::Int, tire_pars::Dict{String,Dict{String,Float64}}, tires::Vector{Tuple{String,Int}})
    
    k_1_lin_array = [tire_pars[x[1]]["k_1_lin"] for x in tires]
    k_0_array = [tire_pars[x[1]]["k_0"] for x in tires]
    age_array = [x[2] for x in tires]

    # get number of stints
    no_stints = length(tires)

    # set up problem matrices (P = H and q = f in quadprog)
    P = Diagonal(0.5 .* k_1_lin_array .* 2)  # * 2 because of standard form
    q = (0.5 .+ age_array) .* k_1_lin_array .+ k_0_array

    G = -Matrix{Float64}(I, no_stints, no_stints)  # minimum 1 lap per stint
    h = fill(-1.0, no_stints)

    A = ones(1, no_stints)  # sum of stints must equal total number of laps
    b = [tot_no_laps]

    # ------------------------------------------------------------------------------------------------------------------
    # SET UP SOLVER SPECIFIC PROBLEM -----------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # create integer design variables
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:no_stints], Int)

    # set up problem using objective and constraints
    @objective(model, Min, 0.5 * dot(x, P * x) + dot(q, x))
    @constraint(model, G * x .<= h)
    @constraint(model, A * x .== b)

    # ------------------------------------------------------------------------------------------------------------------
    # CALL SOLVER ------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # solve the problem and retrieve solution
    JuMP.optimize!(model)
    if JuMP.has_values(x)
        stint_lengths = round.(Int, value.(x))
    else
        # no solution found
        stint_lengths = nothing
    end

    return x
end


opt_strategy_basic (generic function with 1 method)

In [33]:
tire_pars = Dict(
    "soft" => Dict("k_1_lin" => 0.02, "k_0" => 0.8),
    "medium" => Dict("k_1_lin" => 0.015, "k_0" => 0.7),
    "hard" => Dict("k_1_lin" => 0.1, "k_0" => 0.6)
)
tires = [("soft", 0), ("medium", 2), ("hard", 5)]
tot_no_laps = 53

result = opt_strategy_basic(tot_no_laps, tire_pars, tires)



Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-21
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xff67b69c
Model has 3 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 1e+00]
  QObjective range [1e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 3 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 184.9050000

Root relaxation: objective 5.267920e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes  

LoadError: MethodError: no method matching has_values(::Vector{VariableRef})
[0mClosest candidates are:
[0m  has_values([91m::Model[39m; result) at ~/.julia/packages/JuMP/vuP7I/src/variables.jl:1061

In [39]:
tire_pars = Dict(
    "soft" => Dict("k_1_lin" => 0.02, "k_0" => 0.8),
    "medium" => Dict("k_1_lin" => 0.015, "k_0" => 0.7),
    "hard" => Dict("k_1_lin" => 0.1, "k_0" => 0.6)
)
tires = [("soft", 0), ("medium", 2), ("hard", 5)]
tot_no_laps = 53

k_1_lin_array = [tire_pars[x[1]]["k_1_lin"] for x in tires]
    k_0_array = [tire_pars[x[1]]["k_0"] for x in tires]
    age_array = [x[2] for x in tires]

    # get number of stints
    no_stints = length(tires)

    # set up problem matrices (P = H and q = f in quadprog)
    P = Diagonal(0.5 .* k_1_lin_array .* 2)  # * 2 because of standard form
    q = (0.5 .+ age_array) .* k_1_lin_array .+ k_0_array

    G = -Matrix{Float64}(I, no_stints, no_stints)  # minimum 1 lap per stint
    h = fill(-1.0, no_stints)

    A = ones(1, no_stints)  # sum of stints must equal total number of laps
    b = [tot_no_laps]

    # ------------------------------------------------------------------------------------------------------------------
    # SET UP SOLVER SPECIFIC PROBLEM -----------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # create integer design variables
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:no_stints], Int)

    # set up problem using objective and constraints
    @objective(model, Min, 0.5 * dot(x, P * x) + dot(q, x))
    @constraint(model, G * x .<= h)
    @constraint(model, A * x .== b)

    # ------------------------------------------------------------------------------------------------------------------
    # CALL SOLVER ------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # solve the problem and retrieve solution
    #JuMP.optimize!(model)



Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-21


1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1] + x[2] + x[3] = 53.0

In [40]:
@time begin
    status = optimize!(model)
end
println("Objective value: ", JuMP.objective_value(model))
println("x[1] = ", JuMP.value(x[1]))
println("x[2] = ", JuMP.value(x[2]))
println("x[3] = ", JuMP.value(x[3]))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xff67b69c
Model has 3 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 1e+00]
  QObjective range [1e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 3 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 184.9050000

Root relaxation: objective 5.267920e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In